In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf
from datasets import load_dataset
import wave
import pyaudio
import PyPDF2

In [ ]:
# Define key component
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [ ]:
def read_page(num):
    pdf_file = open('a.pdf', 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text = pdf_reader.pages[num].extract_text()
    pdf_file.close()
    return text

def speech_vector(text):
    inputs = processor(text=text, return_tensors="pt")
    embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
    speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)
    speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)
    return speech

def read_speech(path):
    wav_file = wave.open(path, "rb")
    p = pyaudio.PyAudio()
    stream = p.open(format=p.get_format_from_width(wav_file.getsampwidth()),
                    channels=wav_file.getnchannels(),
                    rate=wav_file.getframerate(),
                    output=True)
    data = wav_file.readframes(1024)
    while data:
        stream.write(data)
        data = wav_file.readframes(1024)
    stream.stop_stream()
    stream.close()
    p.terminate()

# Process

In [ ]:
text = read_page(15)
print(text)
speech = speech_vector(text[0:600])
sf.write("speech.wav", speech.numpy(), samplerate=16000)
read_speech("speech.wav")

In [ ]:
text = read_page(15)

N = len(text)//600
reminder_len = len(text) - N*600
parts = [] 
for i in range(N):
    sample = text[i*600:(i+1)*600]
    parts.append(sample)
reminder = text[len(text)-reminder_len:]
parts.append(reminder)

for part in parts:
    speech = speech_vector(part)
    sf.write("speech.wav", speech.numpy(), samplerate=16000)
    read_speech("speech.wav")